In [1]:
import sys
sys.path.insert(1, '../')
from FrenetFDA.utils.smoothing_utils import VectorBSplineSmoothing
from FrenetFDA.utils.Frenet_Serret_utils import solve_FrenetSerret_ODE_SE, solve_FrenetSerret_ODE_SO, centering, find_best_rotation
import FrenetFDA.utils.visualization as visu
from FrenetFDA.utils.Lie_group.SO3_utils import SO3
import numpy as np
from scipy.integrate import cumtrapz, solve_ivp

INFO: Using numpy backend


In [2]:
curv = lambda s : 2*np.cos(2*np.pi*s) + 5
tors = lambda s : 2*np.sin(2*np.pi*s) + 1

def theta(s):
    if isinstance(s, int) or isinstance(s, float):
        return np.array([curv(s), tors(s)])
    elif isinstance(s, np.ndarray):
        return np.vstack((curv(s), tors(s))).T
    else:
        raise ValueError('Variable is not a float, a int or a NumPy array.')


grid = np.linspace(0,1,100)
Z0 = np.eye(4)
Q0 = np.eye(3)

In [7]:
Q = solve_FrenetSerret_ODE_SO(theta, grid, Q0)
noisy_Q_10 = SO3.random_point_fisher(n_samples=len(grid), K=100, mean_directions=Q)
noisy_Q_5 = SO3.random_point_fisher(len(grid), K=5, mean_directions=Q)

In [8]:
visu.plot_3D([Q[:,:,0], noisy_Q_10[:,:,0], noisy_Q_5[:,:,0]], ['T', 'noisy_T_10', 'noisy_T_5'])
visu.plot_3D([Q[:,:,1], noisy_Q_10[:,:,1], noisy_Q_5[:,:,1]], ['N', 'noisy_N_10', 'noisy_N_5'])
visu.plot_3D([Q[:,:,2], noisy_Q_10[:,:,2], noisy_Q_5[:,:,2]], ['B', 'noisy_B_10', 'noisy_B_5'])

In [9]:
rand_unif = SO3.random_point_uniform(100)
rand_fisher = SO3.random_point_fisher(100, 1)
print(np.linalg.det(rand_fisher[2]))

1.0


In [19]:

visu.plot_3D([rand_unif[:,:,0], rand_fisher[:,:,0]], ['T', 'noisy_T_10'])
visu.plot_3D([rand_unif[:,:,1], rand_fisher[:,:,1]], ['N', 'noisy_N_10'])
visu.plot_3D([rand_unif[:,:,2], rand_fisher[:,:,2]], ['B', 'noisy_B_10'])

In [3]:
Z = solve_FrenetSerret_ODE_SE(theta, grid, Z0, method='Radau')

In [4]:
print(Z.shape)
print(Z[0])
print(Z[50])
visu.plot_3D([Z[:,:3,0], Z[:,:3,1], Z[:,:3,2], Z[:,:3,3]], ['T', 'N', 'B', 'X'])

(100, 4, 4)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[-0.56839936 -0.24434037  0.78564165  0.06296078]
 [ 0.42831932 -0.90315889  0.029002    0.34975506]
 [ 0.70248853  0.35300598  0.61798478  0.1572241 ]
 [ 0.          0.          0.          1.        ]]


In [5]:
Z_bis = solve_FrenetSerret_ODE_SE(theta, grid, Z0, method='Linearized')
print(Z_bis.shape)

(100, 4, 4)


In [6]:
visu.plot_3D([Z_bis[:,:3,0], Z_bis[:,:3,1], Z_bis[:,:3,2], Z_bis[:,:3,3]], ['T', 'N', 'B', 'X'])

In [4]:
Q = solve_FrenetSerret_ODE_SO(theta, grid, Q0)

In [8]:
visu.plot_3D([Q[:,:3,0], Q[:,:3,1], Q[:,:3,2]], ['T', 'N', 'B'])

In [5]:
R = Q[50]
X = Z[:,:3,3]
print(R)
print(X.shape)
X_rot_left = (R @ X.T).T
X_rot_right = X @ R

[[-0.56841501 -0.2443327   0.78563778]
 [ 0.4282807  -0.9031974   0.02898634]
 [ 0.70250676  0.35300676  0.6179957 ]]
(100, 3)


In [6]:
print(R @ X[1], X_rot_left[1])

[-0.00582302  0.00400013  0.00721688] [-0.00582302  0.00400013  0.00721688]


In [7]:
visu.plot_3D([X, X_rot_left, X_rot_right], ['X', 'X_rot_left', 'X_rot_right'])

In [8]:
new_X_right, R_right = find_best_rotation(X_rot_right, X)
new_X_left, R_left = find_best_rotation(X_rot_left, X)
print(R)
print(R_right)
print(R_left)

[[-0.56841501 -0.2443327   0.78563778]
 [ 0.4282807  -0.9031974   0.02898634]
 [ 0.70250676  0.35300676  0.6179957 ]]
[[-0.56839252  0.42830615  0.70248401]
 [-0.24433583 -0.9031688   0.35296759]
 [ 0.78563983  0.02898212  0.61800493]]
[[-0.5683942  -0.24433572  0.78563865]
 [ 0.42830754 -0.90316807  0.02898445]
 [ 0.7024818   0.35296955  0.61800633]]


In [10]:
visu.plot_3D([X, X_rot_right, X_rot_left, new_X_right, new_X_left], ['X', 'X_rot_right', 'X_rot_left', 'new_X_right', 'new_X_left'])

In [11]:
X_cent = centering(X)
X_rot_cent = centering(X_rot_left)
new_X_cent, R = find_best_rotation(X_cent, X_rot_cent)
print(R)

[[-0.56838504  0.42831026  0.70248755]
 [-0.24432389 -0.90316643  0.35298193]
 [ 0.78564896  0.02899516  0.61799272]]


In [12]:
visu.plot_3D([X_cent, X_rot_cent, new_X_cent], ['X', 'X_rot', 'new_X'])

In [15]:
new_X, R = find_best_rotation(X_rot_left, X)
print(R)

[[-0.5683942  -0.24433572  0.78563865]
 [ 0.42830754 -0.90316807  0.02898445]
 [ 0.7024818   0.35296955  0.61800633]]


In [16]:
visu.plot_3D([X, X_rot_left, new_X], ['X', 'X_rot', 'new_X'])